통계기반기법을 사용한 자연어 처리의 문제점이 있어서 개선한 기법이 추론 기반 기법이다.

In [6]:
# 3.1.3 신경망에서의 단어 처리 (3)

import numpy as np

c = np.array([1, 0, 0, 0, 0, 0, 0])  # 입력 - 단어 ID가 0인 단어를 원핫 표현으로 표현 (1행 7열)
W = np.random.randn(7, 3)  # 가중치 - randn() 함수는 정규분포로부터 난수를 샘플링한다는 뜻이다. (n = normal distribution) / np.random.randn(7, 3)은 7x3 난수 행렬을 만들어낸다.
h = np.matmul(c, W) # 중간 노드 - matmul() 함수는 행렬의 곱셈을 수행한다. / h = cW / c는 1x7 행렬이고 W는 7x3 행렬이므로 h는 1x3 행렬이 된다.

print(h) # 출력 - 중간 노드의 값 출력 (가중치의 행벡터 하나를 추출하는 것과 같다.)


## 결론 ##
# 원핫으로 표현된 단어를 가중치와 곱하면 가중치의 행벡터 하나를 '추출'하는 것과 같다.
# 즉, 이것은 비효율적이기 때문에 4.1 word2vec 개선 1절에서 개선할 예정임.


[-0.19100865  0.38212549 -1.1075906 ]


In [ ]:
# 3.2.1 CBOW 모델의 추론 처리(3)

import sys
sys.path.append('..') # 부모 디렉터리의 파일을 가져올 수 있도록 설정 - 교재 코드가 예제 코드를 올려놓은 github 폴더 구조를 따르기 때문
import numpy as np
from common.layers import MatMul  # common 폴더의 layers.py 파일에서 MatMul 클래스를 불러옴

# 샘플 맥락 데이터 (말뭉치 "you say goodbye and i say hello.")
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])   # 입력 - 단어 ID가 0인 단어 you를 원핫 표현으로 표현 (1행 7열)
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])   # 입력 - 단어 ID가 2인 단어 goodbye를 원핫 표현으로 표현 (1행 7열)  /  target은 'say' 이다. ID = 1

# 가중치 초기화 
W_in = np.random.randn(7, 3)   # 가중치 - np.random.randn(7, 3)은 7x3 난수 행렬을 만들어낸다.
W_out = np.random.randn(3, 7)  # 가중치 - np.random.randn(3, 7)은 3x7 난수 행렬을 만들어낸다.

# 계층 생성
in_layer0 = MatMul(W_in)       # 가중치 W_in을 인수로 받아 MatMul 클래스의 객체를 생성
in_layer1 = MatMul(W_in)       # 가중치 W_in을 인수로 받아 MatMul 클래스의 객체를 생성
out_layer = MatMul(W_out)      # 가중치 W_out을 인수로 받아 MatMul 클래스의 객체를 생성

# 순전파
h0 = in_layer0.forward(c0)     # 입력층 0의 순전파
h1 = in_layer1.forward(c1)     # 입력층 1의 순전파
h = 0.5 * (h0 + h1)            # 은닉층 - 두 입력층의 값의 평균을 은닉층의 값 (은닉층 뉴런)
s = out_layer.forward(h)       # 출력층 - 은닉층의 값의 순전파

print(s)


##========= 결론 =========##
# CBOW 모델의 추론 처리 과정에서 MatMul 계층을 사용하여 출력층에서 각 단어의 점수를 출력했다.
# 단, 이 CBOW 모델은 활성화 함수를 사용하지 않는 간단한 구성의 신경망이다.

# 3.2.2 CBOW 모델의 학습
# CBOW 모델의 학습에서는 소프트맥스 함수를 이용하여 확률로 변환하고 순전파를 통해 손실을 구한 후, 역전파를 통해 가중치를 갱신한다.
# 이것을 계산 반복해서 가중치를 갱신하면서 최적의 단어의 분산 표현을 얻을 수 있다. 
# word2vec에서는 입력층의 가중치를 단어의 분산 표현으로 이용한다.

[[ 0.0245923  -0.37710326 -0.17606803 -0.96502852 -0.0488573  -0.18920723
   0.12913072]]


In [ ]:
# 3.3.1 맥락과 타깃

import sys
sys.path.append('..') # 부모 디렉터리의 파일을 가져올 수 있도록 설정 - 교재 코드가 예제 코드를 올려놓은 github 폴더 구조를 따르기 때문
from common.util import preprocess, create_co_matrix, cos_similarity # common 폴더의 util.py 파일에서 preprocess, create_co_matrix, cos_similarity 함수를 불러옴

text = 'You say goodbye and I say hello.'           # 말뭉치 - 텍스트 데이터
corpus, word_to_id, id_to_word = preprocess(text)   # common 폴더 내의 utils.py에 정의된 전처리 함수 preprocess() - 말뭉치를 전처리 (단어 ID로 변환)

print(corpus)    # [0 1 2 3 4 1 5 6] - 말뭉치를 단어 ID로 변환한 결과

[0 1 2 3 4 1 5 6]


In [ ]:
print(id_to_word)  # {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'} - 단어 ID를 단어로 변환한 결과

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [ ]:
# 3.3.1 맥락과 타깃(2) - 4주차 코드 분석 과제


# 맥락과 타깃을 만드는 함수 구현
def create_contexts_target(corpus, window_size=1):  # window size는 특정 단어를 중심으로 고려하는 주변 단어들의 범위를 의미
    target = corpus[window_size:-window_size]       # target은 말뭉치의 양 끝을 제외한 단어들 ([0, 1, 2, 3, 4, 1, 5, 6] 에서 1:-1 슬라이싱 => [1, 2, 3, 4, 1, 5])  
    contexts = []                                   # 맥락을 저장할 빈 리스트 생성

    for idx in range(window_size, len(corpus)-window_size): # 말뭉치의 단어 수만큼 반복 window_size는 
        cs = []                                             # 반복문 내에서 맥락을 저장할 빈 리스트 생성 (이후 contexts 리스트에 추가 예정)
        for t in range(-window_size, window_size + 1):      # 윈도우 크기만큼 반복, -window_size ~ window_size
            if t == 0:                                      # target 자기 자신은 제외                 
                continue
            cs.append(corpus[idx + t])                      # target이 아닌 단어에 대하여 맥락에 해당 단어 추가
        contexts.append(cs)                                 # 맥락 리스트에 맥락 추가
    
    return np.array(contexts), np.array(target)             # 맥락과 타깃을 반환

context, target = create_contexts_target(corpus, window_size=1)   # window_size=1로 맥락과 타깃 생성

print(context)  # [[0 2] [1 3] [2 4] [3 1] [4 5] [1 6]] - 맥락 출력 결과

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]


In [ ]:
print(target)  # [1 2 3 4 1 5] - 타깃 출력 결과

[1 2 3 4 1 5]


In [ ]:
# 3.3.2 원핫 표현으로 변환 - 맥락과 타깃을 원핫 표현으로 변환 (p.25)

import sys
sys.path.append('..') # 부모 디렉터리의 파일을 가져올 수 있도록 설정 - 교재 코드가 예제 코드를 올려놓은 github 폴더 구조를 따르기 때문
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'           # 말뭉치 - 텍스트 데이터
corpus, word_to_id, id_to_word = preprocess(text)   # common 폴더의 utils.py에 정의된 전처리 함수 preprocess() - 말뭉치를 전처리 (단어 ID로 변환)

context, target = create_contexts_target(corpus, window_size=1)  # 맥락과 타깃 생성

vocab_size = len(word_to_id)  # 어휘 수
target = convert_one_hot(target, vocab_size)  # target을 원핫 표현으로 변환
contexts = convert_one_hot(context, vocab_size)  # context를 원핫 표현으로 변환

print(target)   # 원핫 표현으로 변환된 target 출력


[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]


In [ ]:
print(contexts)  # 원핫 표현으로 변환된 context 출력

[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]


-------------- 3.4 CBOW 모델 구현 ppt 27페이지는 다음주 5주차에 계속